<a href="https://colab.research.google.com/github/gurmu/AutoKeras-image-classification/blob/master/Autokeras_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autokeras

     |████████████████████████████████| 81kB 3.4MB/s 


In [2]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc0


  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc0) to /tmp/pip-req-build-uetx45ie
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-uetx45ie
  Running command git checkout -q 5d16a5a9cd883fe6366d523a5e59b72cfefb7f77
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc0-cp36-none-any.whl size=89059 sha256=68124eb4cb4f45ff32430303435987d113bad96daa9b0c88b386ad17ac016c55
  Stored in directory: /tmp/pip-ephem-wheel-cache-b8byfpw7/wheels/ab/97/20/917008aafbd47b823fb0bc43989739b1195c682036641a49d7
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=0ea6827514f7e3dff304970dcf7684001232b0cb0cc9cda720b4ad21e0ace720
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4]


In [4]:
# Initialize the image classifier.
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Epoch 1/10
1500/1500 [==============================] - 128s 85ms/step - loss: 0.1770 - accuracy: 0.9467 - val_loss: 0.0710 - val_accuracy: 0.9796
Epoch 2/10
1500/1500 [==============================] - 126s 84ms/step - loss: 0.0789 - accuracy: 0.9750 - val_loss: 0.0510 - val_accuracy: 0.9862
Epoch 3/10
1500/1500 [==============================] - 126s 84ms/step - loss: 0.0610 - accuracy: 0.9806 - val_loss: 0.0438 - val_accuracy: 0.9877
Epoch 4/10
1500/1500 [==============================] - 127s 85ms/step - loss: 0.0556 - accuracy: 0.9819 - val_loss: 0.0421 - val_accuracy: 0.9887
Epoch 5/10
1500/1500 [==============================] - 128s 85ms/step - loss: 0.0479 - accuracy: 0.9848 - val_loss: 0.0450 - val_accuracy: 0.9874
Epoch 6/10
1500/1500 [==============================] - 129s 86ms/step - loss: 0.0442 - accuracy: 0.9851 - val_loss: 0.0439 - val_accuracy: 0.9883
Epoch 7/10
1500/1500 [==============================] - 127s 85ms/step - loss: 0.0375 - accuracy: 0.9882 - val_loss: 0

INFO:tensorflow:Oracle triggered exit
1875/1875 [==============================] - 161s 86ms/step - loss: 0.1614 - accuracy: 0.9513
[[7]
 [2]
 [1]
 ...
 [4]
 [5]
 [6]]
313/313 [==============================] - 6s 18ms/step - loss: 0.0629 - accuracy: 0.9781
[0.06293244659900665, 0.9781000018119812]


In [5]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)

In [6]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)

In [7]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=1)
clf.fit(x_train, y_train, epochs=10)

Epoch 1/10
1251/1251 [==============================] - 2679s 2s/step - loss: 0.2832 - accuracy: 0.9230 - val_loss: 0.6353 - val_accuracy: 0.9207
Epoch 2/10
1251/1251 [==============================] - 2789s 2s/step - loss: 0.1025 - accuracy: 0.9731 - val_loss: 0.2302 - val_accuracy: 0.9344
Epoch 3/10
1251/1251 [==============================] - 2756s 2s/step - loss: 0.0992 - accuracy: 0.9742 - val_loss: 0.0980 - val_accuracy: 0.9733
Epoch 4/10
1251/1251 [==============================] - 2652s 2s/step - loss: 0.1066 - accuracy: 0.9718 - val_loss: 0.0752 - val_accuracy: 0.9790
Epoch 5/10
1251/1251 [==============================] - 2652s 2s/step - loss: 0.0799 - accuracy: 0.9785 - val_loss: 0.0745 - val_accuracy: 0.9800
Epoch 6/10
1251/1251 [==============================] - 2662s 2s/step - loss: 0.0749 - accuracy: 0.9794 - val_loss: 0.2515 - val_accuracy: 0.9459
Epoch 7/10
1251/1251 [==============================] - 2654s 2s/step - loss: 0.0601 - accuracy: 0.9837 - val_loss: 0.1697 -

INFO:tensorflow:Oracle triggered exit
1563/1563 [==============================] - 3291s 2s/step - loss: 0.2351 - accuracy: 0.9346


In [14]:
clf.export_model()